In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('ford.csv')

In [3]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


## Encoding

In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


In [5]:
def auto_label_encode(df):
    df = df.copy()
    encoders = {}

    for col in df.select_dtypes(include='object'):
      
        df[col] = le.fit_transform(df[col])
        encoders[col] = le

    return df, encoders


In [6]:
enc_df , encoders= auto_label_encode(df)

In [7]:
enc_df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5,2017,12000,0,15944,4,150,57.7,1.0
1,6,2018,14000,1,9083,4,150,57.7,1.0
2,6,2017,13000,1,12456,4,150,57.7,1.0
3,5,2019,17500,1,10460,4,145,40.3,1.5
4,5,2019,16500,0,1482,4,145,48.7,1.0


## Data split

In [8]:
train, test = train_test_split(enc_df, test_size=0.2)

In [9]:
X_train = train.drop(columns=['price'])
y_train = train['price']

X_test = test.drop(columns=['price'])
y_test = test['price']



## Scaling

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [11]:
num_vars =['year','mileage','tax','mpg']
num_vars

['year', 'mileage', 'tax', 'mpg']

In [12]:
scaler.fit(X_train[num_vars])

X_train[num_vars] = scaler.transform(X_train[num_vars])
X_test[num_vars] = scaler.transform(X_test[num_vars])

In [13]:
X_train.head()

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
10838,5,0.343750,1,0.038228,4,0.250000,0.248066,1.0
15083,13,0.328125,0,0.249512,4,0.258621,0.096133,1.5
84,13,0.296875,1,0.141711,0,0.250000,0.185083,2.0
1711,5,0.328125,1,0.168872,4,0.258621,0.239779,1.1
14797,6,0.359375,1,0.045817,4,0.258621,0.217127,1.0


In [14]:
X_train.shape

(14372, 8)

### we have 8 input features for neural network
ie. 8 columns (14372, 8)

## Neural Network with Keras Functional API

#### Basically 
diff is that no Sequential function is to be used, we define input and output layer manually outside and pass that in .Model()

model = keras.Model(inputs=inputs, outputs=outputs)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
# input layer
inputs = layers.Input(shape=(8,))

# hidden layers
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(10, activation='relu')(x)

# output layer
outputs = layers.Dense(1, activation='relu')(x)



In [17]:
# model funtion call

model = keras.Model(inputs=inputs , outputs = outputs)

In [21]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,209 (118.01 KB)

 Trainable params: 10,069 (39.33 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,140 (78.68 KB)

In [18]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)


In [19]:
model.fit(X_train , y_train, batch_size=32, epochs=100)

Epoch 1/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 885us/step - loss: 82740336.0000 - mae: 7194.9736    
Epoch 2/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - loss: 22505020.0000 - mae: 3510.0286
Epoch 3/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - loss: 19565586.0000 - mae: 3291.2002
Epoch 4/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - loss: 16611858.0000 - mae: 3062.6582
Epoch 5/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - loss: 14087546.0000 - mae: 2842.0435
Epoch 6/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 915us/step - loss: 11952813.0000 - mae: 2615.5898  
Epoch 7/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 890us/step - loss: 10142023.0000 - mae: 2394.8372  
Epoch 8/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - loss: 8947363.0000 - mae: 2251.9167
Epoch 9/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8279311.0000 - mae: 2177.7617  
Epoch 10/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 7878848.0000 - mae: 2128.9268    
Epoch 11/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/s

## Evaluation

In [20]:
mse, mae = model.evaluate(x=X_test, y= y_test)
print(f'Mean Squared Error : {mse}\n')
print(f'Mean Absolute Error : {mae}')

113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 2598940.7500 - mae: 1105.9880
Mean Squared Error : 2598940.75

Mean Absolute Error : 1105.988037109375
